# Importing the Library
1. `from codes.utils.device import device` returns `CUDA` is NVIDIA GPU is available for computation else returns `CPU` if NVIDIA is not available
2. `from codes.models.FNO import TensorizedFNO` is used to load trained FNO models for inference
3. `from codes.models.CNO import CompressedCNO` is used to load trained CNO models for inference
4. `from codes.data.dataset import LidDrivenDataset` is used to load datasets from `npz` format for `x` and `y` mappings for the model
5. `from codes.utils.visualization import plot_ldc_like` is used to plot the `y`, `y_predicted` and `|y_predicted - y|`

In [1]:
import os
import yaml
import torch
import torch.nn as nn
from codes.utils.device import device
from codes.models.FNO import TensorizedFNO
from codes.models.CNO import CompressedCNO
from codes.data.dataset import LidDrivenDataset
from codes.utils.visualization import plot_ldc_like

Using CUDA for tensor operations


# Defining the function to read the configuration YAML file
This function would read the `file_name` YAML file located in the folder `folder_path`

In [2]:
def read_yaml_file(folder_path, file_name):
    """Reads a YAML file and returns the data."""
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r') as file:
        try:
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")
            return None

# Defining the function to calculate the different type of Mean Square Error
### Define further

In [3]:
def all_metrics_validation_set(y, yhat, resolution):

  half_res = int(resolution/2)

  mse = nn.MSELoss()

  mses = torch.zeros(7) # we are outputing 7 validation metrics

  #mse full
  mses[0] = mse(y,yhat)

  #mse u,v
  mses[1] = mse(y[:,0:2,:,:], yhat[:,0:2,:,:])

  #mse p
  mses[2] = mse(y[:,2,:,:], yhat[:,2,:,:])

  #mse near object u, v
  if resolution == 128:
    mses[3] = mse(y[:,0:2,25:104,25:104], yhat[:,0:2,25:104,25:104])
  elif resolution == 256:
    mses[3] = mse(y[:,0:2,51:206,51:206], yhat[:,0:2,51:206,51:206])
  else:
    mses[3] = mse(y[:,0:2,102:410,102:410], yhat[:,0:2,102:410,102:410])

  #mse near object p
  if resolution == 128:
    mses[4] = mse(y[:,2,25:104,25:104], yhat[:,2,25:104,25:104])
  elif resolution == 256:
    mses[4] = mse(y[:,2,51:206,51:206], yhat[:,2,51:206,51:206])
  else:
    mses[4] = mse(y[:,2,102:410,102:410], yhat[:,2,102:410,102:410])

  #mse cd
  mses[5] = mse(y[:,3,:half_res,:], yhat[:,3,:half_res,:])

  #mse cl
  mses[6] = mse(y[:,3,half_res:,:], yhat[:,3,half_res:,:])
  
  #return
  return mses

# Determine the path to the model weights

First we would locate the weights and the location of the configuration files

In [4]:
!ls experiments

cno_1  fno_1


We would like to run inference for CNO. So we will use the folder `cno_1`. The confugaration file found below:

In [5]:
!ls experiments/cno_1

checkpoints  config.yaml  dataset  log.txt  plots


Checkpoint files are found below:

In [6]:
!ls experiments/cno_1/checkpoints

100.pth  200.pth  300.pth  400.pth


The configuration data is imported. This configuration file is used to define the CNO Model that would be loaded

In [7]:
config_data = read_yaml_file('experiments/cno_1/', "config.yaml")

# Loading the weights of 400th Epoch of the trained CNO Model

We would want to clear the cache in the cuda before loading the model if NVIDIA GPU is available

In [8]:
torch.cuda.empty_cache()

Initializing the model using the configuration data

In [9]:
CNO_trained= CompressedCNO(in_dim = config_data['in_dim'], out_dim = config_data['out_dim'], 
                          N_layers = config_data['N_layers'], in_size = config_data['in_size'], 
                          out_size = config_data['out_size']).to(device)

Loading the checkpoint into the model

In [10]:
CNO_trained.load_checkpoint(save_name="400", save_folder='experiments/cno_1/checkpoints')

# Loading the dataset from the `npz` files

In [ ]:
dataset_npz = LidDrivenDataset(file_path_x= config_data['file_path_x'], 
                           file_path_y = config_data['file_path_y'])

# Loading the dataset from the `pt` files saved during training

In [ ]:
dataset_pt = torch.load('experiments/cno_1/dataset/val.pt')

# Performing MSE Calculations with the `dataset_pt`

In [ ]:
# Set the model to evaluation mode
CNO_trained.eval()

# Disable gradient calculation to save memory and speed up computations
with torch.no_grad():
    # Initialize a tensor to store the sum of MSEs for each output
    sum_mse = torch.zeros(7)
    
    # Create a data loader for the validation set
    val_loader = torch.utils.data.DataLoader(dataset_pt, batch_size=5, shuffle=True)
    
    # Iterate over batches in the validation set
    for batch in val_loader:
        # Move the input and target tensors to the device (e.g., GPU)
        inputs, targets = batch[0].to(device), batch[1].to(device)
        
        # Pass the inputs through the model to get the outputs
        outputs = CNO_trained.forward(inputs)
        
        # Calculate the MSE for each output using the all_metrics_validation_set function
        mses = all_metrics_validation_set(targets, outputs, 128)
        
        # Accumulate the MSEs for each output
        sum_mse += mses
    
    # Calculate the average MSE for each output by dividing the sum by the total number of samples
    calculated_mse = sum_mse / len(val_loader.dataset)

# Print the configuration data and the calculated MSEs
print("The Confugaration for CNO:", '\n', config_data, '\n \n', "Calculated MSE:", '\n', calculated_mse)

# Ploting the `y`, `y_predicted` and `|y_predicted - y|` with   MSE Calculations with the `npz` files

In [ ]:
# Set the model to evaluation mode
CNO_trained.eval()

# Disable gradient calculation to save memory and speed up computations
with torch.no_grad():
    # We will use the dataset with the index 10 for this computation.
    id = 10
    
    # Retrieve input data and targets from the dataset
    inputs, targets = dataset_npz[id]
    
    # Perform a forward pass through the model
    outputs = CNO_trained.forward(inputs.unsqueeze(0).to(device))
    
    # Visualize the output and save it in the folder `experiments/cno_1/plots` with the name `CNO_1_{id}.png`
    plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), 
                  outputs.cpu().numpy(), 0, 
                  os.path.join('experiments/cno_1/plots', f'CNO_1_{id}.png'))

# The Plot have been saved at `experiments/cno_1/plots/CNO_1_10.png`
![CNO Output](experiments/cno_1/plots/CNO_1_10.png)

# Repeating for FNO

Calculating the MSE

In [ ]:
# Release any unoccupied cached memory on the GPU
torch.cuda.empty_cache()

# Read configuration data from a YAML file
config_data = read_yaml_file('experiments/fno_1', "config.yaml")

# Create an instance of the FNO model with configuration parameters
FNO_trained = TensorizedFNO(n_modes=config_data['n_modes'], in_channels=config_data['in_channels'], 
                             out_channels=config_data['out_channels'], hidden_channels=config_data['hidden_channels'], 
                             projection_channels=config_data['projection_channels'], n_layers=config_data['n_layers']).to(device)

# Load a pre-trained checkpoint for the FNO model
FNO_trained.load_checkpoint(save_name="400", save_folder='experiments/fno_1/checkpoints')

# Create an instance of the LidDrivenDataset with the data loaded in it
dataset_npz = LidDrivenDataset(file_path_x=config_data['file_path_x'], 
                               file_path_y=config_data['file_path_y'])

# Set the model to evaluation mode
FNO_trained.eval()

# Disable gradient calculation to save memory and speed up computations
with torch.no_grad():
    # Initialize a tensor to store the sum of MSEs for each output
    sum_mse = torch.zeros(7)
    
    # Create a data loader for the validation set
    val_loader = torch.utils.data.DataLoader(dataset_npz, batch_size=5, shuffle=True)
    
    # Iterate over batches in the validation set
    for batch in val_loader:
        # Move the input and target tensors to the device (e.g., GPU)
        inputs, targets = batch[0].to(device), batch[1].to(device)
        
        # Pass the inputs through the model to get the outputs
        outputs = FNO_trained.forward(inputs)
        
        # Calculate the MSE for each output using the all_metrics_validation_set function
        mses = all_metrics_validation_set(targets, outputs, 128)
        
        # Accumulate the MSEs for each output
        sum_mse += mses
    
    # Calculate the average MSE for each output by dividing the sum by the total number of samples
    calculated_mse = sum_mse / len(val_loader.dataset)

# Print the configuration data and the calculated MSEs
print("The Confugaration for CNO:", '\n', config_data, '\n \n', "Calculated MSE:", '\n', calculated_mse)

Plotting the data

In [ ]:
# Set the model to evaluation mode
FNO_trained.eval()

# Disable gradient calculation to save memory and speed up computations
with torch.no_grad():
    # We will use the dataset with the index 10 for this computation.
    id = 10
    
    # Retrieve input data and targets from the dataset
    inputs, targets = dataset_npz[id]
    
    # Perform a forward pass through the model
    outputs = FNO_trained.forward(inputs.unsqueeze(0).to(device))
    
    # Visualize the output and save it in the folder `experiments/fno_1/plots` with the name `FNO_1_{id}.png`
    plot_ldc_like(targets.unsqueeze(0).cpu().numpy(), 
                  outputs.cpu().numpy(), 0, 
                  os.path.join('experiments/fno_1/plots', f'FNO_1_{id}.png'))

# The Plot for the FNO have been saved at `experiments/fno_1/plots/FNO_1_10.png`
![CNO Output](experiments/fno_1/plots/FNO_1_10.png)